In [1]:
import pandas as pd

data = pd.read_csv("./jobs.csv")
# data['Job Title'].count()
data.head()

,Unnamed: 0,Job Salary,Job Experience Required,Key Skills,Role Category,Functional Area,Industry,Job Title
0,0,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager
1,1,Not Disclosed by Recruiter,2 - 5 yrs,pre sales| closing| software knowledge| clien...,Retail Sales,"Sales , Retail , Business Development","IT-Software, Software Services",Sales Executive/Officer
2,2,Not Disclosed by Recruiter,0 - 1 yrs,Computer science| Fabrication| Quality check|...,R&D,"Engineering Design , R&D","Recruitment, Staffing",R&D Executive
3,3,"2,00,000 - 4,00,000 PA.",0 - 5 yrs,Technical Support,Admin/Maintenance/Security/Datawarehousing,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Support Engineer
4,4,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Programming & Design,IT Software - QA & Testing,"IT-Software, Software Services",Testing Engineer


In [2]:
#Renaming unanmmed columns
data.rename(columns={'Unnamed: 0': 'job_id'}, inplace=True)

In [3]:
#Now let’s have a look if the dataset contains any null values or not:

data.isnull().sum()

job_id                     0
Job Salary                 0
Job Experience Required    0
Key Skills                 0
Role Category              0
Functional Area            0
Industry                   0
Job Title                  0
dtype: int64

In [4]:
data["pos_com_city_empType_jobDesc"] = data['Job Title'].map(str) + " " + data["Industry"] +" "+ data["Functional Area"]+ " "+data['Role Category']+" "+data['Key Skills']
data.pos_com_city_empType_jobDesc.head()
print(data.pos_com_city_empType_jobDesc[3])

Technical Support Engineer IT-Software, Software Services IT Software - Application Programming , Maintenance Admin/Maintenance/Security/Datawarehousing  Technical Support


In [5]:
!pip install scikit-learn --upgrade

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# load preprocessed dataset
preprocessed_data = data

# create a TfidfVectorizer object
vectorizer = TfidfVectorizer(stop_words='english')

# fit the vectorizer on the 'pos_com_city_empType_jobDesc' column
vectorizer.fit(preprocessed_data['pos_com_city_empType_jobDesc'])

TfidfVectorizer(stop_words='english')

In [7]:
# Transform the job descriptions into a matrix of TF-IDF features
tfidf_matrix = vectorizer.transform(preprocessed_data['pos_com_city_empType_jobDesc'])

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Get user query as input
user_query = "software testing, regression testing, functional testing, black-box testing, white-box testing, user acceptance testing, integration testing, system testing, performance testing, load testing, stress testing, security testing, agile testing methodologies, defect management, test case design, test plan development, test strategy, test reporting, continuous integration, Jenkins, Selenium, Appium, JMeter, Postman, Git, and SQL databases"

# Transform the user query using the fitted TfidfVectorizer object
user_query_tfidf = vectorizer.transform([user_query]) 

# Compute cosine similarity between user query and all job descriptions
cosine_similarities = cosine_similarity(user_query_tfidf, tfidf_matrix).flatten()

# Add the cosine similarities to the preprocessed dataset as the 'target' column
preprocessed_data['target'] = cosine_similarities

In [9]:

# Train a RandomForestClassifier on the preprocessed dataset
from sklearn.ensemble import RandomForestRegressor

# Define X and y variables for training the model
X = tfidf_matrix
y = preprocessed_data['target']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Create a RandomForestClassifier object
rfc = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on the training data
rfc.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [11]:

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predict target values using the trained RFR model
y_pred = rfc.predict(X_test)

# Calculate RMSE and MAE scores
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)

# Evaluate the accuracy of the model on the testing data
accuracy = rfc.score(X_test, y_test)
print("Accuracy:", accuracy)

RMSE: 0.007130001330425001
MAE: 0.001953960104413175
Accuracy: 0.9905330716683152


In [12]:
import joblib

# save the trained model to disk
joblib.dump(rfc, 'trained_model.sav')


['trained_model.sav']

In [19]:
import joblib

# load the saved model from disk
loaded_model = joblib.load('trained_model.sav')

# get user input
user_input = input("Enter your query: ")

# generate the TF-IDF matrix for the user input
user_tfidf = vectorizer.transform([user_input])

# make prediction using the loaded model
predicted_similarities = loaded_model.predict(user_tfidf)
print(predicted_similarities)

similarity_min = predicted_similarities - 0.001
similarity_max = predicted_similarities + 0.001

# Sort the jobs based on the predicted cosine similarity values and return top 5 recommendations
top_n_jobs = 5
recommendations = preprocessed_data.loc[(preprocessed_data['target'] >= float(similarity_min)) &(preprocessed_data['target'] <= float(similarity_max))].sort_values('target', ascending=False).head(top_n_jobs)

# recommendations = preprocessed_data.loc[preprocessed_data['target'].isin(predicted_similarities)].sort_values('target', ascending=False).head(top_n_jobs)

# Print the recommended jobs
print(recommendations[['pos_com_city_empType_jobDesc']])

Enter your query:  QA, Testing


[0.52874946]
                           pos_com_city_empType_jobDesc
3159  Quality Assurance/Quality Control Manager IT-S...
9518  Testing Engineer IT-Software, Software Service...
